In [74]:
%matplotlib notebook
import csv
from scipy.io import loadmat
import numpy as np
import scipy.optimize as opt
from pandas.io.parsers import read_csv
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from sklearn.preprocessing import PolynomialFeatures

In [75]:
data = loadmat('ex4data1.mat')
y = data['y']
x = data['X']
unos = np.full((5000,1),1)
weights = loadmat('ex4weights.mat')
theta1, theta2 = weights['Theta1'], weights['Theta2']
x = np.append(unos, x, axis=1)
params = np.concatenate((np.ravel(theta1),np.ravel(theta2)))

In [76]:
def sigmoide(z):
    s = np.dot(z,-1)
    e = np.exp(s)
    d = 1 + e
    return 1/d

In [77]:
def propagacion (x,th):
    unos = np.full((len(x),1),1)
    z = np.dot(x,np.transpose(th))
    return np.append(unos,sigmoide(z), axis = 1)

In [78]:
def testeo(y, f):
    acertados = 0
    for i in range(len(y)):
        if np.argmax(f[i])+1 == y[i]:
            acertados +=1
    return acertados, acertados/len(y)*100

In [79]:
def norm(y,num_etiqueta):
    y_norm = np.zeros((len(y),num_etiqueta))
    for i in range(len(y)):
        y_norm[i][y[i]-1]= 1
    return np.transpose(y_norm)

In [80]:
y_norm = norm(y,10)
g = np.copy(propagacion(propagacion(x,theta1),theta2)[:,1:])


In [106]:
g.shape

(5000, 10)

In [93]:
p1 =np.dot(y_norm,np.log(g))
p2 = np.dot((1-y_norm),np.log(1-g))
p1.shape

(10, 10)

In [103]:
def coste(theta1,theta2,x,y,num_etiqueta):
    y = norm(y,num_etiqueta)
    g = np.copy(propagacion(propagacion(x,theta1),theta2)[:,1:])
    return np.sum(np.dot(-y,np.log(g)) + np.dot(-(1-y),np.log(1-g)))/len(x)
coste(theta1,theta2,x,y,10)

89.36616413827161

In [104]:
def backprop (params_rn, num_entradas, num_ocultas, num_etiquetas, x, y, reg = 0.1 ):
    theta1 = np.reshape(params_rn[:num_ocultas*(num_entradas+1)], (num_ocultas, (num_entradas+1)))
    theta2 = np.reshape(params_rn[num_ocultas*(num_entradas+1):], (num_etiquetas, (num_ocultas+1)))
    cost = coste(theta1,theta2,x,y,num_etiquetas)
    return cost

In [105]:
backprop(params,theta1.shape[1]-1,theta2.shape[1]-1,10,x,y)

89.36616413827161

5000